In [ ]:
import re
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# re: https://www.cnblogs.com/tina-python/p/5508402.html

In [ ]:
match = re.match(r'(?P<filename>[a-zA-Z0-9_\.]*[^:]):(?P<line>\d+)', 'RuleEngineLog.h:44')
# match = re.match(r'(?P<function>[a-zA-Z0-9_]*)', 'clips_logd')
match.groupdict()

In [ ]:
# 08-31 10:06:09.703  2882  3544 V homebrain: 2034181200 | RuleEngineLog.h:44 | clips_logd | 4 | ("creaate rule context " rul-cloud-virtualmode)
hb_tag=r'homebrain'
p_dt=r'(?P<datetime>\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3})'
p_id1=r'(?P<pthreadid>\d+)'
p_id2=r'(?P<pid>\d+)'
p_id3=r'(?P<kernelpid>\d+)'
p_fnl=r'(?P<filename>[\w\.]*):(?P<line>\d+)'
p_func=r'(?P<function>\w*)'
p_loglevel=r'(?P<level>\d{1})'
p_message=r'(?P<message>.*[^$])'
hb_re = re.compile(p_dt+'\s+'+p_id1+'\s+'+p_id2+' V '+hb_tag+': '+p_id3+' \| '+p_fnl+' \| '+p_func+' \| '+p_loglevel+' \| '+p_message+'$')

ret = hb_re.search('08-31 10:06:09.703  2882  3544 V homebrain: 2034181200 | RuleEngineLog.h:44 | clips_logd | 4 | (\"creaate rule context \" rul-cloud-virtualmode)')
print(ret)
print(ret.groupdict())

def parser(file):
    try:
        with open(file) as f:
            for line in f:
                res = hb_re.search(line)
                if res is None:
                    continue
                gdict = res.groupdict()
                # dt = datetime.strptime(gdict['datetime'], '%m-%d %H:%M:%S.%f').replace(year=2018)
                # print(dt)
                print(gdict)

    except IOError as e:
        raise e

In [ ]:
parser('./logcat.log.01')